
# Projet 2

In [1]:
# import des modules usuels
import numpy as np
import pandas as pd

# options d'affichage
pd.set_option("display.min_rows", 16)

In [2]:
# chargement et traitement des données
geo = pd.read_csv("correspondance-code-insee-code-postal.csv",
                   sep=';',
                   usecols=range(11),
                   index_col="Code INSEE")
geo = geo.sort_index()
geo[['Latitude', 'Longitude']] = geo["geo_point_2d"].str.extract("(.*), (.*)").astype(float)
geo.head()

,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,Latitude,Longitude
Code INSEE,,,,,,,,,,,,
01001,01400,L'ABERGEMENT-CLEMENCIAT,AIN,RHONE-ALPES,Commune simple,242.0,1565.0,0.8,"46.1534255214, 4.92611354223","{""type"": ""Polygon"", ""coordinates"": [[[4.926273...",46.153426,4.926114
01002,01640,L'ABERGEMENT-DE-VAREY,AIN,RHONE-ALPES,Commune simple,483.0,912.0,0.2,"46.0091878776, 5.42801696363","{""type"": ""Polygon"", ""coordinates"": [[[5.430089...",46.009188,5.428017
01004,01500,AMBERIEU-EN-BUGEY,AIN,RHONE-ALPES,Chef-lieu canton,379.0,2448.0,13.4,"45.9608475114, 5.3729257777","{""type"": ""Polygon"", ""coordinates"": [[[5.386190...",45.960848,5.372926
01005,01330,AMBERIEUX-EN-DOMBES,AIN,RHONE-ALPES,Commune simple,290.0,1605.0,1.6,"45.9961799872, 4.91227250796","{""type"": ""Polygon"", ""coordinates"": [[[4.895580...",45.996180,4.912273
01006,01300,AMBLEON,AIN,RHONE-ALPES,Commune simple,589.0,602.0,0.1,"45.7494989044, 5.59432017366","{""type"": ""Polygon"", ""coordinates"": [[[5.614854...",45.749499,5.594320


**Projet maison n° 1**

- Ajoutez une colonne 'CP Ville' avec le Code postal + un espace + et le nom de la Ville.
- Ecrivez une fonction qui détermine la commune la plus proche d'un point à partir de sa latitude et sa longitude.
- Ajoutez une fonction de conversion pour pouvoir utiliser la première fonction avec un GPS (degrés, minutes, secondes).

In [3]:
geo['CP Ville'] =  geo['Code Postal'] + " " + geo['Commune']

In [4]:
# fonction recherche de ville
def ville(lat, long):
    geo['X'] = abs(geo['Latitude'] - lat) + abs(geo['Longitude'] - long)
    res = min(geo['X'])
    return geo.loc[geo["X"] == res]

In [5]:
# conversion degrés, minutes, secondes => décimal
def dms2dec(deg, mn, sec):
    return deg + mn / 60 + sec / 3600

In [6]:
# on applique la fonction à une coordonnée tirée au hasard
# init du random
np.random.seed(0)

a, b = 41.5, 51.1  # latitude min et max de la France métropolitaine
lat = (b - a) * np.random.random() + a
a, b = -5.1, 9.5  # longitude min et max de la France métropolitaine
long = (b - a) * np.random.random() + a

ville(lat, long)

,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,Latitude,Longitude,CP Ville,X
Code INSEE,,,,,,,,,,,,,,
71044,71330,BOSJEAN,SAONE-ET-LOIRE,BOURGOGNE,Commune simple,202.0,1862.0,0.3,"46.7696786287, 5.34727924927","{""type"": ""Polygon"", ""coordinates"": [[[5.345021...",46.769679,5.347279,71330 BOSJEAN,0.006583


In [7]:
# à partir de coordonnées GPS précises
ville(dms2dec(48, 42, 52), dms2dec(2, 14, 45))

,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,Latitude,Longitude,CP Ville,X
Code INSEE,,,,,,,,,,,,,,
91477,91120,PALAISEAU,ESSONNE,ILE-DE-FRANCE,Sous-préfecture,118.0,1168.0,30.3,"48.7146765876, 2.22881488083","{""type"": ""Polygon"", ""coordinates"": [[[2.259350...",48.714677,2.228815,91120 PALAISEAU,0.017251


**Projet maison n° 2**

La colonne "geo_shape" comporte des chaines de catactères au format JSON. Elles représentent les formes géométriques des communes qui sont soit des polygones soit composées de plusieurs polygones.

- Utiliser la librairie Python json pour parser la colonne "geo_shape".
- Donner le décompte des valeurs accédées avec la clé "type".
- Donner le décompte des longueurs des listes accédées avec la clé "coordinates".
- Quelle commune est la plus complexe géométriquement ?
- Quelle commune est la seconde la plus complexe géométriquement ?
- Quelles sont les villes qui sont de type "Polygon" mais dont la longueur des listes accédées avec la clé "coordinates" vaut 2 ?
- Pour ces villes vérifier que le premier polygone contient bien le second (enclave). NB : installer la librairie shapely, utiliser la classe Polygon de shapely.geometry. Sur Windows shapely peut nécessiter d'installer la dll "geos_c.dll" dans le répertoire "Library/bin" de votre environnement Python.

Utiliser la librairie Python json pour parser la colonne "geo_shape".

In [9]:
frames = list()
for i in geo.index.tolist() :
    y = geo.loc[i, "geo_shape"]
    d = json.loads(y)
    d["index"] = i
    d["longueur"] = len(d['coordinates'])
    frames.append(pd.DataFrame([d]))
result = pd.concat(frames)
result.set_index('index',inplace = True)
result

,type,coordinates,longueur
index,,,
01001,Polygon,"[[[4.92627342692396, 46.1200517234555], [4.921...",1
01002,Polygon,"[[[5.430089291292173, 45.98275997279625], [5.4...",1
01004,Polygon,"[[[5.386190824025674, 45.930927620632], [5.374...",1
01005,Polygon,"[[[4.895580369408855, 45.97991618124606], [4.8...",1
01006,Polygon,"[[[5.614854625053272, 45.734638855121595], [5....",1
01007,Polygon,"[[[5.413533405897647, 45.97889779714948], [5.3...",1
01008,Polygon,"[[[5.321986043355919, 45.92234494663176], [5.3...",1
01009,Polygon,"[[[5.65639346945457, 45.76021095279123], [5.64...",1
...,...,...,...


Donner le décompte des valeurs accédées avec la clé "type".

In [10]:
result['type'].value_counts()

Polygon         36670
MultiPolygon       72
Name: type, dtype: int64

Donner le décompte des longueurs des listes accédées avec la clé "coordinates".

In [11]:
result['longueur'].value_counts()

1    36660
2       80
3        1
4        1
Name: longueur, dtype: int64

Quelle commune est la plus complexe géométriquement ?

In [12]:
l = "".join(result.loc[result['longueur']==max(result['longueur']),:].index.tolist())

geo.loc[geo.index == l,:]

,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,Latitude,Longitude,CP Ville,X
Code INSEE,,,,,,,,,,,,,,
83069,83400,HYERES,VAR,PROVENCE-ALPES-COTE D'AZUR,Chef-lieu canton,60.0,13356.0,54.7,"43.1018713534, 6.18898508469","{""type"": ""MultiPolygon"", ""coordinates"": [[[[6....",43.101871,6.188985,83400 HYERES,9.555725


Quelle commune est la seconde la plus complexe géométriquement ?

In [13]:
n = sorted(np.unique(result['longueur']))

print(n[-2])

l = "".join(result.loc[result['longueur']==n[-2],:].index.tolist())

geo.loc[geo.index == l,:]

3


,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,Latitude,Longitude,CP Ville,X
Code INSEE,,,,,,,,,,,,,,
27701,27100,VAL-DE-REUIL,EURE,HAUTE-NORMANDIE,Chef-lieu canton,23.0,2593.0,13.5,"49.261335704, 1.21147195365","{""type"": ""MultiPolygon"", ""coordinates"": [[[[1....",49.261336,1.211472,27100 VAL-DE-REUIL,1.581253


Quelles sont les villes qui sont de type "Polygon" mais dont la longueur des listes accédées avec la clé "coordinates" vaut 2 ?

In [14]:
result_sauv = result

In [53]:
result = result_sauv

In [54]:
# left join par défaut 
result = result.join(geo[['Commune', 'Département', 'Région', 'CP Ville']])

result_2 = result.loc[(result['longueur']==2) & (result['type']=='Polygon'),:] ; result_2


,type,coordinates,longueur,Commune,Département,Région,CP Ville
index,,,,,,,
02232,Polygon,"[[[3.037897560658703, 49.16921270775551], [3.0...",2,COYOLLES,AISNE,PICARDIE,02600 COYOLLES
05052,Polygon,"[[[6.719912606601575, 44.70392733592357], [6.7...",2,EYGLIERS,HAUTES-ALPES,PROVENCE-ALPES-COTE D'AZUR,05600 EYGLIERS
09042,Polygon,"[[[1.423960819701936, 42.99550854650971], [1.4...",2,LA BASTIDE-DE-SEROU,ARIEGE,MIDI-PYRENEES,09240 LA BASTIDE-DE-SEROU
22209,Polygon,"[[[-2.11237709725211, 48.52896113651634], [-2....",2,PLOUBALAY,COTES-D'ARMOR,BRETAGNE,22650 PLOUBALAY
2A272,Polygon,"[[[8.940487655060174, 41.490088793265336], [8....",2,SARTENE,CORSE-DU-SUD,CORSE,20100 SARTENE
2B049,Polygon,"[[[8.911220447280732, 42.41019895400797], [8.8...",2,CALENZANA,HAUTE-CORSE,CORSE,20214 CALENZANA
48080,Polygon,"[[[3.850531831651621, 44.68745054452225], [3.8...",2,LANGOGNE,LOZERE,LANGUEDOC-ROUSSILLON,48300 LANGOGNE
67486,Polygon,"[[[7.594939251325704, 48.23228567615893], [7.5...",2,SUNDHOUSE,BAS-RHIN,ALSACE,67920 SUNDHOUSE
68078,Polygon,"[[[7.281156117554148, 48.02598189605906], [7.2...",2,EGUISHEIM,HAUT-RHIN,ALSACE,68420 EGUISHEIM


Pour ces villes vérifier que le premier polygone contient bien le second (enclave).

In [58]:
from shapely.geometry import Polygon
result_2c = result_2.copy()
result_2c['enclave'] = Polygon(result_2c['coordinates'][0][0]).contains(Polygon(result_2c['coordinates'][0][1]))
result_2c

,type,coordinates,longueur,Commune,Département,Région,CP Ville,enclave
index,,,,,,,,
02232,Polygon,"[[[3.037897560658703, 49.16921270775551], [3.0...",2,COYOLLES,AISNE,PICARDIE,02600 COYOLLES,True
05052,Polygon,"[[[6.719912606601575, 44.70392733592357], [6.7...",2,EYGLIERS,HAUTES-ALPES,PROVENCE-ALPES-COTE D'AZUR,05600 EYGLIERS,True
09042,Polygon,"[[[1.423960819701936, 42.99550854650971], [1.4...",2,LA BASTIDE-DE-SEROU,ARIEGE,MIDI-PYRENEES,09240 LA BASTIDE-DE-SEROU,True
22209,Polygon,"[[[-2.11237709725211, 48.52896113651634], [-2....",2,PLOUBALAY,COTES-D'ARMOR,BRETAGNE,22650 PLOUBALAY,True
2A272,Polygon,"[[[8.940487655060174, 41.490088793265336], [8....",2,SARTENE,CORSE-DU-SUD,CORSE,20100 SARTENE,True
2B049,Polygon,"[[[8.911220447280732, 42.41019895400797], [8.8...",2,CALENZANA,HAUTE-CORSE,CORSE,20214 CALENZANA,True
48080,Polygon,"[[[3.850531831651621, 44.68745054452225], [3.8...",2,LANGOGNE,LOZERE,LANGUEDOC-ROUSSILLON,48300 LANGOGNE,True
67486,Polygon,"[[[7.594939251325704, 48.23228567615893], [7.5...",2,SUNDHOUSE,BAS-RHIN,ALSACE,67920 SUNDHOUSE,True
68078,Polygon,"[[[7.281156117554148, 48.02598189605906], [7.2...",2,EGUISHEIM,HAUT-RHIN,ALSACE,68420 EGUISHEIM,True
